In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from itertools import permutations
from pyspark.sql.functions import monotonically_increasing_id 
from scipy.sparse import coo_matrix
from google.cloud import storage


In [11]:
#Create the list of files
client = storage.Client()
bucket = client.bucket('bgse-datawarehousing-random-tweets')
json_files =[]
for blob in bucket.list_blobs(prefix=''):
    json_files.append('gs://bgse-datawarehousing-random-tweets/'+blob.name)
json_files

()


In [97]:
#Create a spark dataframe from the files
files = 5
data = spark.read.json(json_files[0:files])

In [12]:
data.printSchema()

NameError: name 'data' is not defined

In [18]:
#Get the tuples of all the possible combinations of hashtags for a given 
#tweet, plus a counter for the number of tweets in which each combination
#appears

tuples = data.rdd \
    .filter(lambda l: ((l.entities is not None) and \
                    (len(l.entities.hashtags) > 1))) \
    .map(lambda l: [i.text.lower() for i in l.entities.hashtags]) \
    .flatMap(lambda l: permutations(l,2)) \
    .map(lambda l: (l,1)) \
    .reduceByKey(lambda a,b: a+b) \
    .map(lambda l: (l[0][0], l[0][1],l[1])) \
    .collect()


NameError: name 'data' is not defined

In [ ]:
#Create a dictionary for unique hashtags
hashtags = list(set([x[0] for x in tuples]))
sorted_hashtags = sorted(hashtags)
dict_hashtags = {sorted_hashtags[i] : i for i in range(0,len(sorted_hashtags))}


In [105]:
#Create rows, columns and values for our sparse matrix
rows = np.array([dict_hashtags[i[0]]for i in tuples])
columns = np.array([dict_hashtags[i[1]]for i in tuples])
count = np.array([i[2] for i in tuples])


<1069x1069 sparse matrix of type '<type 'numpy.int64'>'
	with 5568 stored elements in COOrdinate format>

In [ ]:
#Create sparse matrix
sp_matrix = coo_matrix((count, (rows,columns)), shape = (len(sorted_hashtags),len(sorted_hashtags)))

In [82]:
# Save the matrix in a file
scipy.sparse.save_npz("sp_matrix.npz", sp_matrix, compressed=True)

{u'2019oscarsin5words': 0,
 u'25feb': 1,
 u'alliance': 2,
 u'america': 3,
 u'amerika': 4,
 u'artvsartist': 5,
 u'artvsartist2019': 6,
 u'asambleadelospueblos': 7,
 u'atheist': 8,
 u'austria': 9,
 u'barr': 10,
 u'bbc': 11,
 u'bernietownhall': 12,
 u'bigots': 13,
 u'blockchain': 14,
 u'bonespurs': 15,
 u'bregret': 16,
 u'brent': 17,
 u'brexit': 18,
 u'brexitbetrayal': 19,
 u'brexitbritain': 20,
 u'brexitchaos': 21,
 u'brexiteers': 22,
 u'brexitshambles': 23,
 u'brexitvote': 24,
 u'buildthewall': 25,
 u'bush': 26,
 u'canada': 27,
 u'cartoon': 28,
 u'ccot': 29,
 u'childsexrings': 30,
 u'china': 31,
 u'cleanbrexit': 32,
 u'clinton': 33,
 u'cnn': 34,
 u'comedy': 35,
 u'congress': 36,
 u'conservative': 37,
 u'coquimbo': 38,
 u'corbyn': 39,
 u'corbynsolvesbrexit': 40,
 u'covingtoncatholic': 41,
 u'crazyusa': 42,
 u'ctl': 43,
 u'darwinism': 44,
 u'demand': 45,
 u'democracy': 46,
 u'democrats': 47,
 u'deplorables': 48,
 u'despitebrexit': 49,
 u'disgusting': 50,
 u'disheveled': 51,
 u'doj': 52,
 